# 木町家キッチン シフト最適化ノートブック

このノートブックでは、以下の3ステップでシフト表を作成します：

1. **テンプレート生成** — 指定した年月のシフト表テンプレートExcelを生成
2. **最適化実行** — 希望休（◎）入力済みExcelを読み込み、GAで最適化
3. **結果確認** — 最適化結果のサマリーを表示

---

## 0. 設定

対象の年月を設定してください。

In [ ]:
# ====== ここを変更してください ======
YEAR = 2026
MONTH = 3
# ==================================

from pathlib import Path

OUTPUT_DIR = Path("../output")
OUTPUT_DIR.mkdir(exist_ok=True)

TEMPLATE_PATH = OUTPUT_DIR / f"shift_template_{YEAR}_{MONTH:02d}.xlsx"
RESULT_PATH   = OUTPUT_DIR / f"shift_result_{YEAR}_{MONTH:02d}.xlsx"

print(f"対象: {YEAR}年{MONTH}月")
print(f"テンプレート: {TEMPLATE_PATH}")
print(f"結果出力先:   {RESULT_PATH}")

## 1. テンプレート生成

木町家キッチンのプリセット（5名分の社員情報・土日定休）でテンプレートExcelを生成します。

In [ ]:
from ga_shift.io.template_generator import generate_kimachiya_template

generate_kimachiya_template(str(TEMPLATE_PATH), YEAR, MONTH)
print(f"✅ テンプレート生成完了: {TEMPLATE_PATH}")
print()
print("📝 次のステップ:")
print(f"   1. {TEMPLATE_PATH} をExcelで開く")
print("   2. 各社員の希望休の日に ◎ を入力")
print("   3. 保存して閉じる")
print("   4. 下のセル（Step 2）を実行")

## 2. 最適化実行

◎を入力したテンプレートExcelを読み込み、遺伝的アルゴリズム（GA）でシフトを最適化します。

> **注意**: 先にStep 1で生成したExcelに◎を入力・保存してから実行してください。

In [ ]:
from ga_shift.io.excel_reader import read_shift_input
from ga_shift.agents.conductor import ConductorAgent

# Excel読み込み
shift_input = read_shift_input(str(TEMPLATE_PATH))
print(f"読み込み完了: {shift_input.num_employees}名 × {shift_input.num_days}日")
print()

# 社員情報の確認
for emp in shift_input.employees:
    pref = len(emp.preferred_days_off)
    unavail = len(emp.unavailable_days)
    print(f"  {emp.name}（{emp.employee_type.value}/{emp.section.value}）"
          f" 休日数:{emp.required_holidays} 希望休:{pref} 出勤不可:{unavail}")

print()
print("⏳ GA最適化を実行中...")

# 最適化パイプライン実行
conductor = ConductorAgent()
result = conductor.run_full_pipeline(
    shift_input=shift_input,
    output_path=str(RESULT_PATH),
)

sr = result["shift_result"]
vr = result["validation_report"]

print(f"✅ 最適化完了!")
print(f"   スコア（ペナルティ）: {sr.best_score}")
print(f"   世代数: {sr.generation_count}")
print(f"   結果Excel: {RESULT_PATH}")

## 3. 結果サマリー

最適化結果のシフト表と制約違反を確認します。

In [ ]:
import calendar
import numpy as np

WEEKDAY_JA = ["月", "火", "水", "木", "金", "土", "日"]
MARK = {0: "出", 1: "休", 2: "◎", 3: "×"}

schedule = sr.best_schedule
num_days = shift_input.num_days

# ヘッダー
day_headers = [f"{d}" for d in range(1, num_days + 1)]
wd_headers  = [WEEKDAY_JA[calendar.weekday(YEAR, MONTH, d)] for d in range(1, num_days + 1)]

print(f"{'社員名':　<8}", " ".join(f"{h:>2}" for h in day_headers), " 実休日")
print(f"{'':　<8}", " ".join(f"{h:>2}" for h in wd_headers))
print("-" * (10 + num_days * 3 + 6))

for i, emp in enumerate(shift_input.employees):
    row = schedule[i]
    marks = " ".join(f"{MARK.get(v, '?'):>2}" for v in row)
    holidays = int(np.sum(row > 0))
    print(f"{emp.name:　<8} {marks}  {holidays:>3}")

# キッチン出勤者数
working = np.sum(schedule == 0, axis=0)
print("-" * (10 + num_days * 3 + 6))
print(f"{'出勤者数':　<8}", " ".join(f"{int(w):>2}" for w in working))

In [ ]:
# バリデーション結果
print("=" * 50)
print("バリデーション結果")
print("=" * 50)
print(f"合計ペナルティ: {vr.total_penalty}")
print(f"エラー数: {vr.error_count}")
print(f"警告数:   {vr.warning_count}")
print(f"準拠状態: {'✅ OK' if vr.is_compliant else '⚠️ 違反あり'}")
print()

if vr.constraint_scores:
    print("制約別スコア:")
    for cs in vr.constraint_scores:
        status = "✅" if cs.penalty == 0 else "⚠️"
        print(f"  {status} {cs.name}: {cs.penalty}")

if vr.violations:
    print()
    print("違反詳細:")
    for v in vr.violations[:10]:  # 最大10件表示
        print(f"  [{v.severity}] {v.message}")

---

## 使い方まとめ

| ステップ | 操作 | 備考 |
|:---:|:---|:---|
| 0 | `YEAR` と `MONTH` を設定 | セルを編集して実行 |
| 1 | テンプレート生成セルを実行 | `output/shift_template_YYYY_MM.xlsx` が作成される |
| - | **Excelで◎を入力して保存** | 希望休を入力する唯一の手作業 |
| 2 | 最適化実行セルを実行 | GA実行 → `output/shift_result_YYYY_MM.xlsx` が作成される |
| 3 | 結果サマリーセルを実行 | シフト表と制約違反を確認 |

結果Excelには「GA結果シフト表」と「バリデーション結果」の2シートが含まれます。